In [1]:
import pandas as pd
import os
import glob
import numpy as np
from shutil import copyfile

In [2]:
pd.set_option('display.max_colwidth', 1000)

# Data Merging

In [3]:
df_about = pd.read_csv('../../../data/aboutyou/data.csv', sep=';', encoding='utf-8')
df_fashion = pd.read_csv('../../../data/fashionid/data.csv', sep=';', encoding='utf-8')
df_zalando = pd.read_csv('../../../data/zalando/data.csv', sep=';', encoding='utf-8')

In [4]:
print(df_about.shape)
print(df_fashion.shape)
print(df_zalando.shape)

(40477, 10)
(11857, 10)
(43413, 9)


In [5]:
df_zalando['product_url'] = ''

In [6]:
df_about['data_path'] = 'aboutyou'
df_fashion['data_path'] = 'fashionid'
df_zalando['data_path'] = 'zalando'

In [7]:
df = df_about.append(df_fashion.append(df_zalando))
df['id'] = df['id'].astype(str)
df.shape

(95747, 11)

# Rename categories

In [8]:
def translate_attributes(attr, translations):
    for english, german_list in translations.items():
        if any(german in attr for german in german_list):
            return english

In [9]:
df.category.unique()

array(['blusen-und-tuniken', 'hosen', 'jacken', 'jeans', 'kleider',
       'roecke', 'shirts', 'strick', 'tops', 'blusen', 'pullover-strick',
       'blusen-tuniken', 'jacken-maentel', 'pullover-und-strickjacken'], dtype=object)

In [10]:
category_translation = {
        'tops': ['shirts', 'tops'],
        'pants': ['hosen', 'jeans'],
        'knitwear': ['strick'],
        'jackets': ['jacken'],
        'dresses': ['kleider'],
        'skirts': ['roecke'],
        'blouses': ['blusen'],
        'jumpsuits': ['jumpsuits']}

In [11]:
df['category_en'] = df['category'].apply(
    lambda x: translate_attributes(x, category_translation))

In [12]:
df.groupby('category_en')['category'].apply(lambda x: ','.join(x.unique())).reset_index()

,category_en,category
0,blouses,"blusen-und-tuniken,blusen,blusen-tuniken"
1,dresses,kleider
2,jackets,"jacken,jacken-maentel"
3,knitwear,"strick,pullover-strick,pullover-und-strickjacken"
4,pants,"hosen,jeans"
5,skirts,roecke
6,tops,"shirts,tops"


In [13]:
df['category'] = df['category_en']
df = df.drop('category_en', axis=1)

# Attributes Selection

## Attributes Analysis
Loads the data.csv file and prints the most common attributes (tags). Then creates columns for the selected attributes and converts them to dummy variables.

In [14]:
df.describe()

,img_path,attributes,brand,category,color,id,img_url,model_img_urls,name,product_url,data_path
count,95747,95747,95747,95747,95747,95747,95747,95486,95747,95747,95747
unique,95747,93515,1089,7,9,92220,91857,91645,40264,48974,3
top,jeans/9733029.jpg,"Material & Pflegehinweise, Material Oberstoff: 100% Polyester, Futter: 100% Polyester, Materialkonstruktion: Gehäkelt, Pflegehinweise: Handwäsche, chemische Reinigung möglich, Mehr zu diesem Produkt, Stark transparente Partien, Kragen: Stehkragen, Verschluss: Reißverschluss, Muster: Unifarben, Details: Unterrock, Artikelnummer: TRF21C01O-J11, Modelgröße: Unser Model ist 178 cm groß und trägt Größe 38, Ärmellänge: Ärmellos, Jackenlänge: 40 cm bei Größe 38, Rückenbreite: 31 cm bei Größe 38, Kostenloser Versand & Rückversand, 100 Tage Rückgaberecht",ONLY,tops,blue,IV521C00D-A11,https://cdn.aboutstatic.com/file/88c3ad5d2d588801af19a8b057897a97,"https://cdn.aboutstatic.com/file/316fb99a2c7bb3da037a19e6205bddab, https://cdn.aboutstatic.com/file/9674e396443058c9a7d26bc24a0b6a4f, https://cdn.aboutstatic.com/file/5e732d1ddeed898836cb8f89d9663655, https://cdn.aboutstatic.com/file/9e7f7904e8751e667930f63308c989ae",Bluse,,zalando
freq,1,3,3700,20085,25904,3,4,3,3339,43413,43413


In [15]:
# delete rows that belong to 2 categories
df = df.groupby('id').first().reset_index()

In [16]:
df['tags'] = df['attributes'].apply(lambda x: [t.strip() for t in x.lower().split(',')])

Get count for each of the tags, if it appears in the attributes list of any of the images.

In [17]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

Sort by the most used tags and print

In [18]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df.sort_values('ratio', ascending=False)

In [19]:
tag_df['tag'].unique().tolist()

['material & pflegehinweise',
 '100 tage rückgaberecht',
 'mehr zu diesem produkt',
 'kostenloser versand & rückversand',
 'unifarben',
 'pflegehinweise:\xa0nicht trockner geeignet',
 'maschinenwäsche bei 30°c',
 'passform:\xa0normal',
 'muster:\xa0unifarben',
 'passform: normale passform',
 'schonwaschgang',
 'weicher griff',
 'länge:\xa0normale länge',
 'elastizität: leicht elastisch',
 'rundhals-ausschnitt',
 'ausschnitt:\xa0rundhals',
 'länge: normale länge',
 'elastizität: nicht elastisch',
 'ärmellänge:\xa0langarm',
 'abgesteppter saum/kante',
 'ton-in-ton-nähte',
 'ärmellänge: langarm',
 'jersey',
 'gerader saum',
 'all-over-muster',
 'pflegehinweise:\xa0maschinenwäsche bei 40°c',
 'material oberstoff:\xa0100% baumwolle',
 'taillierter schnitt',
 'lockerer schnitt',
 'materialkonstruktion:\xa0jersey',
 'details:\xa0elastischer bund',
 'baumwolle',
 'gerader schnitt',
 'verschluss:\xa0verdeckter zip-fly',
 'nicht trockner geeignet',
 'leibhöhe:\xa0normal',
 'ärmellänge:\xa0extral

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [20]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Sleeve Length

In [21]:
sleeve_map = (tag_df['tag'].str.contains('ärmel') &
              ~(tag_df['tag'].str.contains('cm|größe')) & 
              (tag_df['ratio'] > 0))

In [22]:
sleeve_list = tag_df[sleeve_map]['tag'].tolist()
df = create_tag_column(df, sleeve_list, 'sleeve_length')

In [23]:
sleeve_translation = {'half': ['dreiviertel', 'halb'],
                      'short': ['kurz', 'viertel'],
                      'long': ['lang'],
                      'sleeveless': ['spaghetti', 'ärmellos']}

In [24]:
df['sleeve_length_en'] = df.loc[~df.sleeve_length.isnull(), 'sleeve_length'].apply(
    lambda x: translate_attributes(x, sleeve_translation))

In [25]:
df.groupby('sleeve_length_en')['sleeve_length'].apply(lambda x: ','.join(x.unique())).reset_index()

,sleeve_length_en,sleeve_length
0,half,"dreiviertelarm,halbarm,raglanärmel in dreiviertel-länge,dreiviertel-ärmel mit geknöpften bündchen,dreiviertel-ärmel mit raffungen,dreiviertel-ärmel,dreiviertel-ärmel mit elastischem einsatz und raffungen,angeschnittene dreiviertel-ärmel,trompetenärmel in dreiviertel-länge,dreiviertel-ärmel mit fixiertem aufschlag,dreiviertel-ärmel mit ausgestellten abschlüssen,angeschnittene ärmel in dreiviertel-länge,dreiviertel-ärmel mit schnürung an den abschlüssen,dreiviertel-ärmel mit geschlitzten abschlüssen,leicht ausgestellte dreiviertel-ärmel,dreiviertel-ärmel mit geknöpftem abschluss,ärmel in dreiviertel-länge,raglanärmel in dreiviertel-länge mit schnürung an den abschlüssen,ballonärmel in dreiviertel-länge,glockenärmel in dreiviertel-länge,dreiviertel-ärmel mit elastischen abschlüssen,ärmel in dreiviertel-länge mit fixiertem aufschlag"
1,long,"langarm,extralanger arm,extra lange ärmel,lange ärmel mit geknöpften bündchen,lange ärmel,lange ärmel mit überschnittenen schultern,lange ärmel mit elastischen bündchen,lange raglanärmel,lange ärmel mit 1-knopf-manschetten,lange ärmel mit geschlitzten abschlüssen,lange ärmel mit fixiertem aufschlag,lange ärmel mit geknöpften abschlüssen,lange ärmel aus chiffon,lange ärmel mit elastischen,lange ärmel mit 2-knopf-manschetten,lange ärmel mit gerippten abschlüssen,lange ärmel mit elastischen abschlüssen,lange ärmel mit ausgestellten abschlüssen,lange ärmel mit gesmokten abschlüssen,lange ärmel mit schnürung an den abschlüssen,lange trompetenärmel,lange ärmel mit geknöpftem abschluss,lange ärmel mit knopfverschluss,lange ärmel mit kontraststreifen"
2,short,"viertelarm,kurzarm,extrakurzer arm,kurze ärmel mit gerippten abschlüssen,kurze ärmel,sehr kurze raglanärmel,kurze raglanärmel,kurze ärmel mit fixiertem aufschlag,sehr kurze ärmel,kurze reißverschlüsse an den ärmelabschlüssen,kurze flügelärmel,kurze trompetenärmel,lange ärmel mit kurzem reißverschluss an den abschlüssen,sehr kurze ärmel mit fixiertem aufschlag,kurze glockenärmel"
3,sleeveless,"ärmellos,spaghettiträger,ärmelloser schnitt,ärmelloses unterkleid aus satin,ärmelloser,ärmelloses unterkleid,ärmelloses,ärmelloses futter"


In [26]:
df['sleeve_length'] = df['sleeve_length_en']
df = df.drop('sleeve_length_en', axis=1)

### Length

In [27]:
length_map = (tag_df['tag'].str.startswith('länge') &
              ~(tag_df['tag'].str.contains('cm|größe')) & 
              (tag_df['ratio'] > 0))

In [28]:
length_list = tag_df[length_map]['tag'].tolist()
df = create_tag_column(df, length_list, 'length')

In [29]:
length_translation = {'normal': ['normal', 'mittel'],
                      'knee': ['knielang'],
                      'short': ['kurz', 'oberschenkel'],
                      '3-4': ['7/8', '3/4', 'waden', 'knöchel'],
                      'long': ['lang']}

In [30]:
df['length_en'] = df.loc[~df.length.isnull(), 'length'].apply(
    lambda x: translate_attributes(x, length_translation))

In [31]:
df.groupby('length_en')['length'].apply(lambda x: ','.join(x.unique())).reset_index()

,length_en,length
0,3-4,"7/8 länge,knöchellang,wadenlang,7/8-lang,3/4-lang,3/4 länge"
1,knee,knielang
2,long,"lang/maxi,langer schnitt,lang,extra lang"
3,normal,"normale länge,mittel"
4,short,"kurz/mini,extra kurz,kurz,kurzer schnitt,oberschenkellang"


In [32]:
df['length'] = df['length_en']
df = df.drop('length_en', axis=1)

### Neckline

In [33]:
neckline_map = (tag_df['tag'].str.contains('ausschnitt') &
                tag_df['ratio'] > 0)

In [34]:
neckline_list = tag_df[neckline_map]['tag'].tolist()
df = create_tag_column(df, neckline_list, 'neckline')

In [35]:
neckline_translation = {'v': ['v-ausschnitt'],
                        'round': ['rund'],
                        'lined': ['eingefasst'],
                        'back': ['rücken'],
                        'wide': ['weit', 'carmen', 'u-boot'],
                        'deep': ['tief']
                     }

In [36]:
df['neckline_en'] = df.loc[~df.neckline.isnull(), 'neckline'].apply(
    lambda x: translate_attributes(x, neckline_translation))

In [37]:
df.groupby('neckline_en')['neckline'].apply(lambda x: ','.join(x.unique())).reset_index()

,neckline_en,neckline
0,back,"rückenausschnitt,tiefer rückenausschnitt"
1,deep,tiefer ausschnitt/dekolleté
2,lined,eingefasster ausschnitt
3,round,"rundhals-ausschnitt,rundhals,tiefer rundhals,rundhalsausschnitt,rundhalsausschnitt mit gerolltem abschluss,gerippter rundhalsausschnitt,breiter rundhalsausschnitt,weiter rundhalsausschnitt,gerollter rundhalsausschnitt,rundhalsausschnitt mit aussparung,rundhalsausschnitt mit gelegten falten,rundhalsausschnitt mit ziersteinbesatz,elastischer rundhalsausschnitt,rundhalsausschnitt mit zierperlenbesatz,rundhalsausschnitt mit geknöpftem schlüssellochausschnitt,rundhalsausschnitt mit gerolltem saum,rundhalsausschnitt mit raffungen,rundhalsausschnitt mit eingesetztem schlüssellochausschnitt,rundhalsausschnitt mit schnürung,rundhalsausschnitt mit hakenverschluss,rundhalsausschnitt mit metall-applikation,rundhalsausschnitt mit offenem abschluss,gerippter rundhalsausschnitt mit kontraststreifen,rundhalsausschnitt mit cut out,rundhalsausschnitt mit zierborte,rundhalsausschnitt mit cut outs,gerippter rundhalsausschnitt in kontrastfarbe,rundhalsausschnitt in wickeloptik,rundhalsausschnitt mit fe..."
4,v,"tiefer v-ausschnitt,v-ausschnitt,abgerundeter v-ausschnitt,rundhalsausschnitt mit eingesetztem v-ausschnitt,stehkragen mit eingesetztem v-ausschnitt,eingesetzter v-ausschnitt,v-ausschnitt in wickeloptik,gerippter v-ausschnitt,v-ausschnitt mit schluppe zum binden,v-ausschnitt mit volantbesatz,v-ausschnitt mit schnürung,v-ausschnitt auf vorder- und rückseite,v-ausschnitt auf der rückseite,eingesetzter v-ausschnitt mit schnürung,stehkragen mit eingesetztem v-ausschnitt in wickeloptik,seitlich versetzter v-ausschnitt,eingesetzter v-ausschnitt in wickeloptik,v-ausschnitt mit gewelltem abschluss,rundhalsausschnitt mit eingesetztem v-ausschnitt in wickeloptik,umlegekragen mit eingesetztem v-ausschnitt,tiefer v-ausschnitt auf vorder- und rückseite,rundhalsausschnitt und eingesetztem v-ausschnitt,abgerundeter v-ausschnitt mit gelegten falten,gerollter v-ausschnitt"
5,wide,"u-boot-ausschnitt,weiter ausschnitt,carmen-ausschnitt,weite ärmelausschnitte,u-boot,carmen,u-boot ausschnitt,elastischer carmenausschnitt,gesmokter carmenausschnitt,umgelegter carmenausschnitt"


In [38]:
df['neckline'] = df['neckline_en']
df = df.drop('neckline_en', axis=1)

### Pattern

In [39]:
pattern_map = (tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze|unifarben$') &
               tag_df['ratio'] > 0)

In [40]:
pattern_list = tag_df[pattern_map]['tag'].tolist()
df = create_tag_column(df, pattern_list, 'pattern')

In [41]:
pattern_translation = {'floral': ['blüm', 'floral'],
                       'polkadots': ['punkt'],
                       'stripes': ['streif'],
                       'print': ['all-over', 'meliert', 'print', 'camouflage', 'kariert', 'paisley'],
                       'lace': ['spitze'],
                       'unicolors': ['unifarben']
                      }

In [42]:
df['pattern_en'] = df.loc[~df.pattern.isnull(), 'pattern'].apply(
    lambda x: translate_attributes(x, pattern_translation))

In [43]:
df.groupby('pattern_en')['pattern'].apply(lambda x: ','.join(x.unique())).reset_index()

,pattern_en,pattern
0,floral,"geblümt/floral,geblümt,oberteil aus floraler spitze,florale stickerei,florale stickereien,florale spitze,einsatz aus floraler spitze,florale häkelspitze,oberteil aus floraler spitze mit pailletten-besatz,oberteil aus mesh mit floralen stickereien,mesh mit floralen stickereien,florales muster,florale stickereien mit ziersteinbesatz,florale stickereien mit pailletten-besatz,verstärktes bustier aus floraler spitze,florales jacquardmuster,florale zierborten,oberteil aus floraler häkelspitze,oberteil mit floralen stickereien,einsätze aus floraler spitze,florale stickereien am oberteil,locker fallendes oberteil aus floraler spitze,besatz aus floraler häkelspitze,ausgestellter rockteil mit floralem muster,florales allover-muster,floraler print,floraler spitzenbesatz"
1,lace,"spitze,spitzenbesatz am saum,abgerundete spitze,häkelspitze,spitzenbesatz,taillenpasse aus häkelspitze,oberteil aus spitze,oberteil aus spitze mit pailletten-besatz,oberteil aus häkelspitze,ornamentale häkelspitze,stehkragen aus spitze,lochspitze,besatz aus häkelspitze,oberteil mit spitzenbesatz,einsätze aus lochspitze,einsatz aus spitze,zierborten aus häkelspitze,zierborten aus lochspitze,einsatz aus lochspitze,einsatz aus spitze auf der rückseite,einsatz aus häkelspitze,oberteil aus spitze mit ziersteinbesatz,stark transparenter spitzenbesatz,stark transparente spitzenpartie,stark transparente spitzenpartien,stark transparente spitze,stark transparenter spitzeneinsatz"
2,polkadots,"gepunktet,punktemuster,strukturiertes punktemuster,eingewebtes punktemuster,feines punktemuster"
3,print,"all-over-muster,meliert,print,paisley-muster,fotoprint,animal print,kariert,paisley,paisleymuster,camouflage-muster,camouflage"
4,stripes,"gestreift,seitliche streifen,seitenstreifen,streifenmuster,nadelstreifen,streifenmuster in melangeoptik,zierstreifen,streifenmuster mit effektgarn,wattierte cups und silikonstreifen,silikonstreifen für sicheren halt,silikonstreifen für besseren halt,seitliche zierstreifen,feines streifenmuster,kontraststreifen,blockstreifen,eingestricktes streifenmuster,seitliche kontraststreifen,zierstreifen mit lochmuster,saum mit kontraststreifen,silikonstreifen und wattierte cups,seitliche logo-streifen,kontraststreifen an den abschlüssen,gerippter rundhalsausschnitt mit kontraststreifen,zierstreifen aus lochspitze,eingewebtes streifenmuster,wechselndes streifenmuster,logo-streifen,ringelstreifen,zierstreifen aus häkelspitze,seitliche kontraststreifen mit effektgarn,seitlicher kontraststreifen,kontraststreifen am saum,kurze kontraststreifen aus samt,strukturiertes streifenmuster,blockstreifenmuster,zierstreifen an den seiten,abgerundeter kontrastsaum mit streifenmuster,lange ärmel mit kontrasts..."
5,unicolors,unifarben


In [44]:
df['pattern'] = df['pattern_en']
df = df.drop('pattern_en', axis=1)

### Fit

In [45]:
fit_map = (tag_df['tag'].str.contains('passform| schnitt') &
          tag_df['ratio'] > 0) 

In [46]:
fit_list = tag_df[fit_map]['tag'].tolist()
df = create_tag_column(df, fit_list, 'fit')

In [47]:
fit_translation = {'normal': ['gerade', 'normal', 'straight', 'regular'],
                   'loose': ['locker', 'weit', 'loose', 'oversized', 'wide'],
                   'tight': ['tailliert', 'tailiert', 'skinny', 'ausgestellt',
                             'körper', 'figur', 'eng', 'slim', 'schmal', 
                             'ausgestellt', 'jegging']
                  }

In [48]:
df['fit_en'] = df.loc[~df.fit.isnull(), 'fit'].apply(
    lambda x: translate_attributes(x, fit_translation))

In [49]:
df.groupby('fit_en')['fit'].apply(lambda x: ','.join(x.unique())).reset_index()

,fit_en,fit
0,loose,"lockerer schnitt,weite passform,weit geschnitten,oversized,lockere passform,loosefit,loose fit,wide leg,weiter schnitt"
1,normal,"gerader schnitt,normale passform,regular,normal,straight leg,regular fit"
2,tight,"taillierter schnitt,skinny,slimfit,ausgestellter schnitt,körpernah,schmal,figurbetonte passform,schmale passform,tailliert,slim fit,leicht taillierter schnitt,figurnaher schnitt,leicht ausgestellter schnitt,körpernahe passform,figurbetonter schnitt,jeggings,körperbetonter schnitt,körpernaher schnitt,eng anliegender schnitt,taillierte passform,tailierter schnitt"


In [50]:
df['fit'] = df['fit_en']
df = df.drop('fit_en', axis=1)

In [51]:
df.fit.unique()

array(['loose', 'tight', nan, 'normal', None], dtype=object)

# Copy all images to new folder

In [52]:
data_folder = '../../../data/fashion_new/'

In [53]:
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)
        
    src_path = os.path.join('../../../data', row['data_path'], row['img_path'])
    
    dst_folder = os.path.join(data_folder, row['category'])
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
        
    dst_path = os.path.join(dst_folder, row['id'] + '.jpg')
    row['img_path'] = row['category'] + '/' + row['id'] + '.jpg'
    
    if not os.path.exists(dst_path):
        copyfile(src_path, dst_path)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [54]:
df = df.drop(['tags', 'data_path'], axis=1)
df = df.fillna(np.nan)

In [55]:
df.to_csv(os.path.join(data_folder, 'data.csv'), sep=';', encoding='utf-8', index=False)

## Create dummies
Turn the created attribute columns into dummy variables.

In [56]:
df.columns

Index(['id', 'img_path', 'attributes', 'brand', 'category', 'color', 'img_url',
       'model_img_urls', 'name', 'product_url', 'sleeve_length', 'length',
       'neckline', 'pattern', 'fit'],
      dtype='object')

In [57]:
df = df.set_index('img_path')

In [58]:
df_dum = pd.get_dummies(df[['category', 'color', 'length', 'pattern', 'neckline', 'sleeve_length']]).reset_index()

In [59]:
df_dum.tail()

,img_path,category_blouses,category_dresses,category_jackets,category_knitwear,category_pants,category_skirts,category_tops,color_beige,color_black,...,neckline_back,neckline_deep,neckline_lined,neckline_round,neckline_v,neckline_wide,sleeve_length_half,sleeve_length_long,sleeve_length_short,sleeve_length_sleeveless
92215,pants/ZX121S001-Q11.jpg,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
92216,jackets/ZX121U001-C11.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
92217,jackets/ZX121U001-Q11.jpg,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
92218,knitwear/ZZLGWA022-J11.jpg,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
92219,tops/ZZLL0K050-Q11.jpg,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [60]:
df_dum.to_csv(os.path.join(data_folder, 'img_attr.csv'), encoding='utf-8', index=False)